In [1]:
import chess
from stockfish import Stockfish

In [2]:
def configure_stockfish_using_skill_level(skill_level):
    stockfish = Stockfish(path="../models/stockfish-windows-x86-64-avx2.exe")
    stockfish.set_skill_level(skill_level)
    return stockfish

def configure_stockfish_using_rating(elo_rating):
    stockfish = Stockfish(path="../models/stockfish-windows-x86-64-avx2.exe")
    stockfish.set_elo_rating(elo_rating)
    return stockfish

def play_game(agent, stockfish, max_moves=50, both_agents_are_stockfish=True, second_stockfish=None):
    board = chess.Board()
    stockfish.set_fen_position(board.fen())
    if both_agents_are_stockfish:
        second_stockfish.set_fen_position(board.fen())
    done = False
    game_moves = 0

    while not done:
        if board.turn:
            # make best move using agent or stockfish
            if both_agents_are_stockfish:
                best_move = second_stockfish.get_best_move()
                best_move = chess.Move.from_uci(best_move)
            else:
                # best move made by your model
                best_move = stockfish.get_best_move()
                best_move = chess.Move.from_uci(best_move)
                best_move = agent.select_action(board, best_move)
            
            board.push(best_move)
            stockfish.set_fen_position(board.fen())
            if both_agents_are_stockfish:
                second_stockfish.set_fen_position(board.fen())
        else:
            stockfish_move = stockfish.get_best_move()
            stockfish_move = chess.Move.from_uci(stockfish_move)
            board.push(stockfish_move)
            stockfish.set_fen_position(board.fen())
            if both_agents_are_stockfish:
                second_stockfish.set_fen_position(board.fen())

        done = board.is_game_over() or game_moves >= max_moves
        game_moves += 1

    result = board.result()  # "1-0", "0-1", "1/2-1/2", "*"
    if result == "1-0":
        return 1  # Agent wins
    elif result == "0-1":
        return -1  # Stockfish wins
    else:
        return 0  # Draw

def evaluate_elo(agent, stockfish_elo_ratings, games_per_level=20, both_agents_are_stockfish=True, second_stockfish_level = 1):
    results = []

    for rating in stockfish_elo_ratings:
        stockfish = configure_stockfish_using_rating(rating)
        if both_agents_are_stockfish:
            second_stockfish = configure_stockfish_using_skill_level(second_stockfish_level)
            
        wins, losses, draws = 0, 0, 0

        for _ in range(games_per_level):
            result = play_game(agent, stockfish, both_agents_are_stockfish=True, second_stockfish=second_stockfish)
            if result == 1:
                wins += 1
            elif result == -1:
                losses += 1
            else:
                draws += 1

        win_rate = (wins + 0.5 * draws) / games_per_level
        results.append({
            "Stockfish Level": rating,
            "Wins": wins,
            "Losses": losses,
            "Draws": draws,
            "Win Rate": win_rate
        })
        print(f"Skill Level {rating}: Wins: {wins}, Losses: {losses}, Draws: {draws}, Win Rate: {win_rate:.2f}")

    return results

stockfish_elo_ratings = [500, 600, 700]
agent = None
results = evaluate_elo(agent, stockfish_elo_ratings, both_agents_are_stockfish=True, second_stockfish_level = 1)

Skill Level 500: Wins: 2, Losses: 0, Draws: 18, Win Rate: 0.55
Skill Level 600: Wins: 0, Losses: 0, Draws: 20, Win Rate: 0.50
Skill Level 700: Wins: 0, Losses: 0, Draws: 20, Win Rate: 0.50
{'Stockfish Level': 500, 'Wins': 2, 'Losses': 0, 'Draws': 18, 'Win Rate': 0.55}
{'Stockfish Level': 600, 'Wins': 0, 'Losses': 0, 'Draws': 20, 'Win Rate': 0.5}
{'Stockfish Level': 700, 'Wins': 0, 'Losses': 0, 'Draws': 20, 'Win Rate': 0.5}


In [3]:
stockfish_elo_ratings = [500, 500, 500]
results = evaluate_elo(agent, stockfish_elo_ratings, both_agents_are_stockfish=True, second_stockfish_level = 1)


Skill Level 500: Wins: 1, Losses: 0, Draws: 19, Win Rate: 0.53
Skill Level 500: Wins: 3, Losses: 1, Draws: 16, Win Rate: 0.55
Skill Level 500: Wins: 2, Losses: 0, Draws: 18, Win Rate: 0.55
